<a href="https://colab.research.google.com/github/ShadmanSobhan/Personal-ChatBot-with-RAG/blob/main/Chatbot_with_RAG.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# === Package installation (quiet, no sqlite3 in pip) ===
!pip install -q groq PyPDF2 sentence-transformers scikit-learn chromadb beautifulsoup4 requests ipywidgets

# Upgrade sentence-transformers quietly (optional)
!pip install -q -U sentence-transformers

# === Imports ===
import os
import sys
import json
import uuid
import hashlib
import sqlite3  # standard lib, no need to install
import requests
from pathlib import Path
from datetime import datetime, timedelta
from typing import List, Dict, Optional, Tuple

import numpy as np
import PyPDF2
import chromadb
from bs4 import BeautifulSoup
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity

# Colab-specific imports with error handling
try:
    from groq import Groq
    from google.colab import files, drive
    import ipywidgets as widgets
    from IPython.display import display, clear_output
except ImportError:
    print("Some packages are missing. Installing ipywidgets...")
    os.system("pip install -q ipywidgets")
    from google.colab import files, drive
    import ipywidgets as widgets
    from IPython.display import display, clear_output


In [ ]:
# Configuration for Colab
class Config:
    GROQ_API_KEY = ""  # Add your Groq API key here
    CHAT_MODEL = "gemma2-9b-it"
    EMBEDDING_MODEL = "all-MiniLM-L6-v2"
    MAX_CONTEXT_MESSAGES = 20
    MAX_PDF_CHUNK_SIZE = 1000
    SEARCH_ENGINE_API_KEY = ""  # Add search API key (optional)
    DATABASE_PATH = "/content/chatbot_data.db"  # Colab content directory
    CHROMA_PATH = "/content/chroma_db"          # Colab content directory
    DRIVE_MOUNT = "/content/drive"              # Google Drive mount point


In [ ]:
class ColabDatabaseManager:
    """Database manager for Colab runtime memory"""

    def __init__(self, db_path: str = Config.DATABASE_PATH):
        self.db_path = db_path
        self.init_database()

    def init_database(self):
        """Initialize SQLite database with required tables"""
        conn = sqlite3.connect(self.db_path)
        cursor = conn.cursor()

        # Conversations table
        cursor.execute('''
            CREATE TABLE IF NOT EXISTS conversations (
                id INTEGER PRIMARY KEY AUTOINCREMENT,
                session_id TEXT NOT NULL,
                role TEXT NOT NULL,
                content TEXT NOT NULL,
                timestamp TEXT NOT NULL,
                message_hash TEXT UNIQUE
            )
        ''')

        # Sessions table
        cursor.execute('''
            CREATE TABLE IF NOT EXISTS sessions (
                session_id TEXT PRIMARY KEY,
                session_name TEXT NOT NULL,
                created_at TEXT NOT NULL,
                last_active TEXT NOT NULL,
                user_id TEXT,
                summary TEXT
            )
        ''')

        # User profiles table
        cursor.execute('''
            CREATE TABLE IF NOT EXISTS user_profiles (
                user_id TEXT PRIMARY KEY,
                name TEXT,
                preferences TEXT,
                interests TEXT,
                created_at TEXT NOT NULL,
                last_updated TEXT NOT NULL
            )
        ''')

        # PDF documents table
        cursor.execute('''
            CREATE TABLE IF NOT EXISTS pdf_documents (
                doc_id TEXT PRIMARY KEY,
                filename TEXT NOT NULL,
                file_path TEXT NOT NULL,
                upload_date TEXT NOT NULL,
                chunk_count INTEGER
            )
        ''')

        conn.commit()
        conn.close()

    def create_session(self, session_name: str, user_id: str = "default") -> str:
        """Create a new conversation session"""
        session_id = str(uuid.uuid4())
        conn = sqlite3.connect(self.db_path)
        cursor = conn.cursor()

        cursor.execute('''
            INSERT INTO sessions (session_id, session_name, created_at, last_active, user_id)
            VALUES (?, ?, ?, ?, ?)
        ''', (session_id, session_name, datetime.now().isoformat(),
              datetime.now().isoformat(), user_id))

        conn.commit()
        conn.close()
        return session_id

    def get_sessions(self, user_id: str = "default") -> List[Dict]:
        """Get all sessions for a user"""
        conn = sqlite3.connect(self.db_path)
        cursor = conn.cursor()

        cursor.execute('''
            SELECT session_id, session_name, created_at, last_active, summary
            FROM sessions WHERE user_id = ?
            ORDER BY last_active DESC
        ''', (user_id,))

        sessions = []
        for row in cursor.fetchall():
            sessions.append({
                'session_id': row[0],
                'session_name': row[1],
                'created_at': row[2],
                'last_active': row[3],
                'summary': row[4]
            })

        conn.close()
        return sessions

    def add_message(self, session_id: str, role: str, content: str):
        """Add a message to conversation history"""
        conn = sqlite3.connect(self.db_path)
        cursor = conn.cursor()

        timestamp = datetime.now().isoformat()
        message_hash = hashlib.md5(f"{session_id}{role}{content}{timestamp}".encode()).hexdigest()

        cursor.execute('''
            INSERT OR IGNORE INTO conversations
            (session_id, role, content, timestamp, message_hash)
            VALUES (?, ?, ?, ?, ?)
        ''', (session_id, role, content, timestamp, message_hash))

        # Update last_active for session
        cursor.execute('''
            UPDATE sessions SET last_active = ? WHERE session_id = ?
        ''', (timestamp, session_id))

        conn.commit()
        conn.close()

    def get_conversation_history(self, session_id: str, limit: int = None) -> List[Dict]:
        """Get conversation history for a session"""
        conn = sqlite3.connect(self.db_path)
        cursor = conn.cursor()

        query = '''
            SELECT role, content, timestamp FROM conversations
            WHERE session_id = ? ORDER BY timestamp ASC
        '''

        if limit:
            query += f' LIMIT {limit}'

        cursor.execute(query, (session_id,))

        messages = []
        for row in cursor.fetchall():
            messages.append({
                'role': row[0],
                'content': row[1],
                'timestamp': row[2]
            })

        conn.close()
        return messages

    def search_conversations(self, query: str, user_id: str = "default") -> List[Dict]:
        """Search through conversation history"""
        conn = sqlite3.connect(self.db_path)
        cursor = conn.cursor()

        cursor.execute('''
            SELECT c.session_id, s.session_name, c.role, c.content, c.timestamp
            FROM conversations c
            JOIN sessions s ON c.session_id = s.session_id
            WHERE s.user_id = ? AND c.content LIKE ?
            ORDER BY c.timestamp DESC
            LIMIT 50
        ''', (user_id, f'%{query}%'))

        results = []
        for row in cursor.fetchall():
            results.append({
                'session_id': row[0],
                'session_name': row[1],
                'role': row[2],
                'content': row[3],
                'timestamp': row[4]
            })

        conn.close()
        return results

    def save_user_profile(self, user_id: str, name: str = None,
                         preferences: Dict = None, interests: List = None):
        """Save or update user profile"""
        conn = sqlite3.connect(self.db_path)
        cursor = conn.cursor()

        timestamp = datetime.now().isoformat()
        prefs_json = json.dumps(preferences) if preferences else None
        interests_json = json.dumps(interests) if interests else None

        cursor.execute('''
            INSERT OR REPLACE INTO user_profiles
            (user_id, name, preferences, interests, created_at, last_updated)
            VALUES (?, ?, ?, ?,
                    COALESCE((SELECT created_at FROM user_profiles WHERE user_id = ?), ?),
                    ?)
        ''', (user_id, name, prefs_json, interests_json, user_id, timestamp, timestamp))

        conn.commit()
        conn.close()

    def get_user_profile(self, user_id: str) -> Dict:
        """Get user profile"""
        conn = sqlite3.connect(self.db_path)
        cursor = conn.cursor()

        cursor.execute('''
            SELECT name, preferences, interests, created_at, last_updated
            FROM user_profiles WHERE user_id = ?
        ''', (user_id,))

        row = cursor.fetchone()
        conn.close()

        if row:
            return {
                'name': row[0],
                'preferences': json.loads(row[1]) if row[1] else {},
                'interests': json.loads(row[2]) if row[2] else [],
                'created_at': row[3],
                'last_updated': row[4]
            }
        return {}


In [ ]:
class ColabPDFProcessor:
    """PDF processor optimized for Colab"""

    def __init__(self):
        print("🔄 Loading embedding model...")
        self.embedding_model = SentenceTransformer(Config.EMBEDDING_MODEL)
        print("🔄 Initializing vector database...")
        self.chroma_client = chromadb.PersistentClient(path=Config.CHROMA_PATH)
        self.collection = self.chroma_client.get_or_create_collection(
            name="pdf_documents",
            metadata={"hnsw:space": "cosine"}
        )
        print("✅ PDF processor ready!")

    def upload_pdf_from_colab(self) -> List[str]:
        """Upload PDF files using Colab file upload"""
        print("📁 Click 'Choose Files' to upload PDF(s)...")
        uploaded = files.upload()

        uploaded_paths = []
        for filename, content in uploaded.items():
            if filename.lower().endswith('.pdf'):
                file_path = f"/content/{filename}"
                with open(file_path, 'wb') as f:
                    f.write(content)
                uploaded_paths.append(file_path)
                print(f"✅ Uploaded: {filename}")
            else:
                print(f"⚠️ Skipped non-PDF file: {filename}")

        return uploaded_paths

    def extract_text_from_pdf(self, pdf_path: str) -> str:
        """Extract text from PDF file"""
        text = ""
        try:
            with open(pdf_path, 'rb') as file:
                pdf_reader = PyPDF2.PdfReader(file)
                for page in pdf_reader.pages:
                    text += page.extract_text() + "\n"
        except Exception as e:
            print(f"❌ Error reading PDF {pdf_path}: {e}")
        return text

    def chunk_text(self, text: str, chunk_size: int = Config.MAX_PDF_CHUNK_SIZE,
                   overlap: int = 200) -> List[str]:
        """Split text into overlapping chunks"""
        if len(text) <= chunk_size:
            return [text]

        chunks = []
        start = 0
        while start < len(text):
            end = start + chunk_size
            if end > len(text):
                end = len(text)

            chunk = text[start:end]
            chunks.append(chunk)

            if end == len(text):
                break

            start = end - overlap

        return chunks

    def add_pdf_to_knowledge_base(self, pdf_path: str, doc_name: str = None) -> str:
        """Add PDF to vector database"""
        if not os.path.exists(pdf_path):
            raise FileNotFoundError(f"PDF file not found: {pdf_path}")

        doc_id = str(uuid.uuid4())
        doc_name = doc_name or os.path.basename(pdf_path)

        print(f"🔄 Processing {doc_name}...")

        # Extract text
        text = self.extract_text_from_pdf(pdf_path)
        if not text.strip():
            raise ValueError("No text extracted from PDF")

        # Create chunks
        chunks = self.chunk_text(text)
        print(f"📄 Created {len(chunks)} chunks")

        # Generate embeddings and store in ChromaDB
        chunk_ids = []
        chunk_texts = []
        metadatas = []

        for i, chunk in enumerate(chunks):
            chunk_id = f"{doc_id}_chunk_{i}"
            chunk_ids.append(chunk_id)
            chunk_texts.append(chunk)
            metadatas.append({
                "doc_id": doc_id,
                "doc_name": doc_name,
                "chunk_index": i,
                "file_path": pdf_path
            })

        # Add to ChromaDB
        print("🔄 Adding to vector database...")
        self.collection.add(
            documents=chunk_texts,
            metadatas=metadatas,
            ids=chunk_ids
        )

        # Save to database
        db = ColabDatabaseManager()
        conn = sqlite3.connect(db.db_path)
        cursor = conn.cursor()
        cursor.execute('''
            INSERT INTO pdf_documents (doc_id, filename, file_path, upload_date, chunk_count)
            VALUES (?, ?, ?, ?, ?)
        ''', (doc_id, doc_name, pdf_path, datetime.now().isoformat(), len(chunks)))
        conn.commit()
        conn.close()

        print(f"✅ PDF processed successfully!")
        return doc_id

    def search_knowledge_base(self, query: str, n_results: int = 5) -> List[Dict]:
        """Search for relevant chunks in knowledge base"""
        try:
            results = self.collection.query(
                query_texts=[query],
                n_results=n_results
            )

            relevant_chunks = []
            if results['documents'] and results['documents'][0]:
                for i, doc in enumerate(results['documents'][0]):
                    metadata = results['metadatas'][0][i]
                    distance = results['distances'][0][i] if 'distances' in results else 0

                    relevant_chunks.append({
                        'content': doc,
                        'doc_name': metadata['doc_name'],
                        'chunk_index': metadata['chunk_index'],
                        'similarity_score': 1 - distance
                    })

            return relevant_chunks
        except Exception as e:
            print(f"❌ Error searching knowledge base: {e}")
            return []

    def get_all_documents(self) -> List[Dict]:
        """Get list of all uploaded documents"""
        db = ColabDatabaseManager()
        conn = sqlite3.connect(db.db_path)
        cursor = conn.cursor()

        cursor.execute('SELECT doc_id, filename, upload_date, chunk_count FROM pdf_documents')
        documents = []
        for row in cursor.fetchall():
            documents.append({
                'doc_id': row[0],
                'filename': row[1],
                'upload_date': row[2],
                'chunk_count': row[3]
            })

        conn.close()
        return documents

In [ ]:
class ColabChatbotInterface:
    """Interactive Colab interface with improved layout"""

    def __init__(self, chatbot):
        self.chatbot = chatbot
        self.conversation_history = []  # Store conversation for display
        self.setup_widgets()

    def setup_widgets(self):
        """Setup interactive widgets with better layout"""

        # Title and status
        self.title_html = widgets.HTML(
            value="<h2>🤖 Enhanced Chatbot with RAG</h2>",
            layout=widgets.Layout(margin='0 0 10px 0')
        )

        self.status_html = widgets.HTML(
            value="<p><span style='color: green'>●</span> Ready to chat!</p>",
            layout=widgets.Layout(margin='0 0 10px 0')
        )

        # Command buttons at the top
        self.new_session_button = widgets.Button(
            description="New Session",
            button_style='success',
            layout=widgets.Layout(width='120px', margin='2px')
        )
        self.sessions_button = widgets.Button(
            description="Sessions",
            button_style='info',
            layout=widgets.Layout(width='100px', margin='2px')
        )
        self.upload_button = widgets.Button(
            description="Upload PDF",
            button_style='warning',
            layout=widgets.Layout(width='110px', margin='2px')
        )
        self.docs_button = widgets.Button(
            description="Documents",
            button_style='',
            layout=widgets.Layout(width='110px', margin='2px')
        )
        self.profile_button = widgets.Button(
            description="Profile",
            button_style='',
            layout=widgets.Layout(width='100px', margin='2px')
        )
        self.clear_button = widgets.Button(
            description="Clear Chat",
            button_style='danger',
            layout=widgets.Layout(width='100px', margin='2px')
        )

        # Chat display area (scrollable)
        self.chat_display = widgets.HTML(
            value="<div style='padding: 15px; background-color: #f8f9fa; border-radius: 8px; min-height: 400px; max-height: 600px; overflow-y: auto; border: 1px solid #dee2e6;'><p style='color: #6c757d; font-style: italic;'>💬 Start chatting! Your conversation will appear here...</p></div>",
            layout=widgets.Layout(margin='10px 0')
        )

        # Input area at the bottom
        self.message_input = widgets.Textarea(
            placeholder="Type your message here... (Shift+Enter for new line, Ctrl+Enter to send)",
            layout=widgets.Layout(width='75%', height='80px', margin='5px 5px 5px 0')
        )

        self.send_button = widgets.Button(
            description="Send 📤",
            button_style='primary',
            layout=widgets.Layout(width='20%', height='80px', margin='5px 0 5px 5px')
        )

        # Search functionality
        self.search_input = widgets.Text(
            placeholder="Search conversation history...",
            layout=widgets.Layout(width='70%', margin='5px 5px 5px 0')
        )

        self.search_button = widgets.Button(
            description="Search 🔍",
            button_style='',
            layout=widgets.Layout(width='25%', margin='5px 0 5px 5px')
        )

        # Event handlers
        self.send_button.on_click(self.send_message)
        self.upload_button.on_click(self.upload_pdf)
        self.new_session_button.on_click(self.new_session)
        self.sessions_button.on_click(self.view_sessions)
        self.docs_button.on_click(self.view_documents)
        self.profile_button.on_click(self.view_profile)
        self.clear_button.on_click(self.clear_chat)
        self.search_button.on_click(self.search_history)

        # Enable keyboard shortcuts
        self.message_input.observe(self.on_key_press, names='value')

    def on_key_press(self, change):
        """Handle keyboard shortcuts (simplified)"""
        # Note: Full keyboard event handling in Jupyter widgets is limited
        # Users can use Ctrl+Enter or click Send button
        pass

    def update_chat_display(self):
        """Update the chat display with conversation history"""
        if not self.conversation_history:
            chat_html = """<div style='padding: 15px; background-color: #f8f9fa; border-radius: 8px;
                          min-height: 400px; max-height: 600px; overflow-y: auto; border: 1px solid #dee2e6;'>
                          <p style='color: #6c757d; font-style: italic;'>💬 Start chatting! Your conversation will appear here...</p></div>"""
        else:
            messages_html = []
            for msg in self.conversation_history:
                msg_content = msg['content'].replace('\n', '<br>')
                timestamp = msg['timestamp'][:16]

                if msg['role'] == 'user':
                    messages_html.append(f"""
                    <div style='margin: 10px 0; padding: 12px; background-color: #e3f2fd; border-radius: 8px; border-left: 4px solid #2196f3;'>
                        <strong>👤 You:</strong> {msg_content}
                        <div style='font-size: 0.8em; color: #666; margin-top: 5px;'>{timestamp}</div>
                    </div>
                    """)
                else:
                    messages_html.append(f"""
                    <div style='margin: 10px 0; padding: 12px; background-color: #f1f8e9; border-radius: 8px; border-left: 4px solid #4caf50;'>
                        <strong>🤖 Assistant:</strong> {msg_content}
                        <div style='font-size: 0.8em; color: #666; margin-top: 5px;'>{timestamp}</div>
                    </div>
                    """)

            chat_html = f"""
            <div style='padding: 15px; background-color: #f8f9fa; border-radius: 8px; min-height: 400px; max-height: 600px; overflow-y: auto; border: 1px solid #dee2e6;'>
                {''.join(messages_html)}
            </div>
            """

        self.chat_display.value = chat_html

    def add_message_to_display(self, role: str, content: str):
        """Add a message to the display history"""
        self.conversation_history.append({
            'role': role,
            'content': content,
            'timestamp': datetime.now().isoformat()
        })
        self.update_chat_display()

    def send_message(self, button=None):
        """Send message handler"""
        message = self.message_input.value.strip()
        if not message:
            return

        # Update status
        self.status_html.value = "<p><span style='color: orange'>●</span> Thinking...</p>"

        # Add user message to display
        self.add_message_to_display('user', message)

        # Clear input
        self.message_input.value = ""

        # Get response
        try:
            response = self.chatbot.generate_response(message)
            self.add_message_to_display('assistant', response)
            self.status_html.value = "<p><span style='color: green'>●</span> Ready to chat!</p>"
        except Exception as e:
            error_msg = f"❌ Error: {str(e)}"
            self.add_message_to_display('assistant', error_msg)
            self.status_html.value = "<p><span style='color: red'>●</span> Error occurred</p>"

    def upload_pdf(self, button):
        """Upload PDF handler"""
        self.status_html.value = "<p><span style='color: orange'>●</span> Uploading PDF...</p>"
        try:
            uploaded_paths = self.chatbot.pdf_processor.upload_pdf_from_colab()
            for path in uploaded_paths:
                doc_id = self.chatbot.pdf_processor.add_pdf_to_knowledge_base(path)
                filename = os.path.basename(path)
                self.add_message_to_display('assistant', f"✅ Successfully uploaded and processed: {filename}")
            self.status_html.value = "<p><span style='color: green'>●</span> PDF uploaded successfully!</p>"
        except Exception as e:
            error_msg = f"❌ Upload failed: {str(e)}"
            self.add_message_to_display('assistant', error_msg)
            self.status_html.value = "<p><span style='color: red'>●</span> Upload failed</p>"

    def new_session(self, button):
        """Create new session"""
        session_name = f"Chat {datetime.now().strftime('%m-%d %H:%M')}"
        session_id = self.chatbot.create_new_session(session_name)
        self.conversation_history = []  # Clear display
        self.update_chat_display()
        self.add_message_to_display('assistant', f"✅ Started new session: {session_name}")
        self.status_html.value = "<p><span style='color: green'>●</span> New session started!</p>"

    def view_sessions(self, button):
        """View all sessions"""
        sessions = self.chatbot.db.get_sessions(self.chatbot.user_id)
        if sessions:
            session_list = [f"📂 Your Sessions ({len(sessions)}):"]
            for i, session in enumerate(sessions[:10], 1):
                indicator = "🔹" if session['session_id'] == self.chatbot.current_session_id else "  "
                session_list.append(f"{i}. {indicator} {session['session_name']} ({session['session_id'][:8]}...)")
                if session['summary']:
                    session_list.append(f"    💭 {session['summary']}")
                session_list.append(f"    📅 {session['last_active'][:16]}")

            session_info = '\n'.join(session_list)
        else:
            session_info = "📂 No sessions found"

        self.add_message_to_display('assistant', session_info)

    def view_documents(self, button):
        """View uploaded documents"""
        docs = self.chatbot.pdf_processor.get_all_documents()
        if docs:
            doc_list = [f"📚 Uploaded Documents ({len(docs)}):"]
            for i, doc in enumerate(docs, 1):
                doc_list.append(f"{i}. 📄 {doc['filename']}")
                doc_list.append(f"    📅 {doc['upload_date'][:16]} | 📊 {doc['chunk_count']} chunks")

            doc_info = '\n'.join(doc_list)
        else:
            doc_info = "📚 No documents uploaded yet. Use 'Upload PDF' button to add documents."

        self.add_message_to_display('assistant', doc_info)

    def view_profile(self, button):
        """View user profile"""
        profile = self.chatbot.user_profile
        profile_info = f"""👤 Your Profile:
   Name: {profile.get('name', 'Not set')}
   Interests: {', '.join(profile.get('interests', [])) or 'Not set'}
   Response Style: {profile.get('preferences', {}).get('response_style', 'Not set')}
   Created: {profile.get('created_at', 'Unknown')[:16] if profile.get('created_at') else 'Unknown'}"""

        self.add_message_to_display('assistant', profile_info)

    def clear_chat(self, button):
        """Clear current chat display"""
        self.conversation_history = []
        self.update_chat_display()
        self.status_html.value = "<p><span style='color: green'>●</span> Chat display cleared!</p>"

    def search_history(self, button):
        """Search conversation history"""
        query = self.search_input.value.strip()
        if not query:
            self.add_message_to_display('assistant', "❌ Please enter a search query")
            return

        results = self.chatbot.db.search_conversations(query, self.chatbot.user_id)
        if results:
            search_results = [f"🔍 Found {len(results)} results for '{query}':"]
            for i, result in enumerate(results[:10], 1):
                search_results.append(f"\n{i}. From {result['session_name']} ({result['timestamp'][:16]})")
                search_results.append(f"   {result['role']}: {result['content'][:100]}...")

            search_info = '\n'.join(search_results)
        else:
            search_info = f"🔍 No results found for '{query}'"

        self.add_message_to_display('assistant', search_info)
        self.search_input.value = ""

    def display(self):
        """Display the improved interface"""

        # Command buttons row
        command_box = widgets.HBox([
            self.new_session_button, self.sessions_button, self.upload_button,
            self.docs_button, self.profile_button, self.clear_button
        ], layout=widgets.Layout(margin='10px 0'))

        # Search box
        search_box = widgets.HBox([
            self.search_input, self.search_button
        ], layout=widgets.Layout(margin='10px 0'))

        # Input area at the bottom
        input_box = widgets.HBox([
            self.message_input, self.send_button
        ], layout=widgets.Layout(margin='10px 0'))

        # Full interface with proper order
        interface = widgets.VBox([
            self.title_html,
            self.status_html,
            command_box,
            search_box,
            self.chat_display,      # Chat conversation in the middle
            input_box               # Input at the bottom
        ])

        display(interface)

        # Welcome message
        welcome_msg = f"""🚀 Enhanced Chatbot with RAG is ready!

Features available:
• 💬 Natural conversation with memory
• 📄 PDF upload and RAG integration
• 🔍 Search conversation history
• 📝 Multiple chat sessions
• 👤 User profile management

Tips:
• Type your message and click Send or use Ctrl+Enter
• Upload PDFs to enhance the chatbot's knowledge
• Use search to find past conversations
• Create new sessions for different topics

{f"Welcome back, {self.chatbot.user_profile['name']}!" if self.chatbot.user_profile.get('name') else ""}"""

        self.add_message_to_display('assistant', welcome_msg)

In [ ]:
# Simplified main chatbot class for Colab
class ColabEnhancedChatbot:
    """Simplified chatbot class for Colab environment"""

    def __init__(self, user_id: str = "colab_user"):
        self.user_id = user_id
        self.current_session_id = None
        self.db = ColabDatabaseManager()
        self.pdf_processor = ColabPDFProcessor()

        # Initialize GROQ client
        if not Config.GROQ_API_KEY:
            raise ValueError("❌ GROQ API key not found. Set it in Config.GROQ_API_KEY")

        self.groq_client = Groq(api_key=Config.GROQ_API_KEY)

        # Prompt user for name
        print("👋 Welcome to the Enhanced Chatbot!")
        user_name = input("Please enter your name to get started: ")
        print(f"✅ Thanks {user_name}! Setting up your profile...")

        print("📝 What are your main interests? (comma separated)")
        interests_input = input("Example: AI, Machine Learning, Science: ")

        # Process interests input
        interests = [interest.strip() for interest in interests_input.split(',') if interest.strip()]

        # Set default interests if none provided
        if not interests:
            print("ℹ️ Using default interests: AI, Machine Learning")
            interests = ["AI", "Machine Learning"]

        print(f"✅ Saving profile with interests: {', '.join(interests)}")

    # Save user profile with name and interests
        self.db.save_user_profile(
            self.user_id,
            name=user_name,
            preferences={"response_style": "casual"},
            interests=interests
        )
        self.user_profile = self.db.get_user_profile(self.user_id)

        # Create initial session
        self.create_new_session("Colab Chat Session")

    def setup_basic_profile(self):
        """Setup basic user profile for Colab"""
        self.db.save_user_profile(
            self.user_id,
            name="Colab User",
            preferences={"response_style": "casual"},
            interests=["AI", "Machine Learning"]
        )
        self.user_profile = self.db.get_user_profile(self.user_id)

    def create_new_session(self, session_name: str = None) -> str:
        """Create a new conversation session"""
        if not session_name:
            session_name = f"Colab Chat {datetime.now().strftime('%Y-%m-%d %H:%M')}"

        session_id = self.db.create_session(session_name, self.user_id)
        self.current_session_id = session_id
        return session_id

    def generate_response(self, user_input: str) -> str:
        """Generate response using GROQ with RAG context"""
        if not self.current_session_id:
            self.create_new_session()

        # Add user message to database
        self.db.add_message(self.current_session_id, "user", user_input)

        # Get conversation history
        history = self.db.get_conversation_history(
            self.current_session_id,
            Config.MAX_CONTEXT_MESSAGES
        )

        # Get RAG context from PDFs
        rag_context = ""
        pdf_results = self.pdf_processor.search_knowledge_base(user_input)
        if pdf_results:
            rag_context = "=== RELEVANT DOCUMENT INFORMATION ===\n"
            for result in pdf_results[:3]:
                rag_context += f"From {result['doc_name']}:\n{result['content'][:500]}...\n\n"

        # Prepare messages for GROQ
        messages = []

        # System message with user profile and context
        system_prompt = "You are a helpful AI assistant with access to uploaded documents."

        if self.user_profile.get('name'):
            system_prompt += f" The user's name is {self.user_profile['name']}."

        if self.user_profile.get('interests'):
            system_prompt += f" User interests: {', '.join(self.user_profile['interests'])}."

        if self.user_profile.get('preferences', {}).get('response_style'):
            style = self.user_profile['preferences']['response_style']
            system_prompt += f" Respond in a {style} style."

        if rag_context:
            system_prompt += f"\n\nRelevant context:\n{rag_context}"

        messages.append({"role": "system", "content": system_prompt})

        # Add conversation history (excluding the current user message to avoid duplication)
        for msg in history[:-1]:  # Exclude the last message we just added
            messages.append({
                "role": msg["role"],
                "content": msg["content"]
            })

        # Add the current user input
        messages.append({"role": "user", "content": user_input})

        try:
            # Call GROQ API
            response = self.groq_client.chat.completions.create(
                model=Config.CHAT_MODEL,
                messages=messages,
                temperature=0.7,
                max_tokens=2048
            )

            assistant_response = response.choices[0].message.content

            # Add assistant response to database
            self.db.add_message(self.current_session_id, "assistant", assistant_response)

            return assistant_response

        except Exception as e:
            error_msg = f"Error generating response: {e}"
            print(error_msg)
            return "I'm sorry, I encountered an error. Please try again."

# Setup and initialization functions for Colab
def setup_colab_environment():
    """Setup the Colab environment"""
    print("🔧 Setting up Colab environment...")

    # Create necessary directories
    os.makedirs(Config.CHROMA_PATH, exist_ok=True)
    print("✅ Directories created")

    print("🚀 Environment ready!")

def start_colab_chatbot():
    """Start the chatbot in Colab"""
    # Check API key
    if not Config.GROQ_API_KEY:
        print("❌ Please set your GROQ API key in Config.GROQ_API_KEY")
        print("Get your key from: https://console.groq.com/")
        return None

    print("🚀 Starting Enhanced Chatbot...")

    # Setup environment
    setup_colab_environment()

    # Initialize chatbot
    try:
        chatbot = ColabEnhancedChatbot()
        interface = ColabChatbotInterface(chatbot)

        print("✅ Chatbot initialized successfully!")
        return interface

    except Exception as e:
        print(f"❌ Initialization failed: {e}")
        return None

# Quick start function
def quick_start():
    """Quick start function for Colab"""
    print("=" * 60)
    print("🤖 ENHANCED CHATBOT WITH RAG - GOOGLE COLAB VERSION")
    print("=" * 60)

    interface = start_colab_chatbot()
    if interface:
        interface.display()
        return interface
    else:
        print("\n📝 Setup Instructions:")
        print("1. Set your GROQ API key: Config.GROQ_API_KEY = 'your_key_here'")
        print("2. Run: quick_start()")
        return None

def demo_usage():
    """Show demo usage examples"""
    print("🎯 Demo Usage Examples:")
    print("\n1. Basic Setup:")
    print("   Config.GROQ_API_KEY = 'your_groq_api_key_here'")
    print("   interface = quick_start()")

    print("\n2. Chat Features:")
    print("   • Type messages in the text area at the bottom")
    print("   • Click 'Send' or use Ctrl+Enter")
    print("   • Upload PDFs for enhanced knowledge")
    print("   • Search through conversation history")

    print("\n3. Advanced Features:")
    print("   • Create multiple chat sessions")
    print("   • View uploaded documents")
    print("   • Manage user profile")
    print("   • Clear chat display when needed")

    print("\n4. UI Layout:")
    print("   • Commands at top")
    print("   • Search bar")
    print("   • Chat conversation in middle (scrollable)")
    print("   • Message input at bottom")


In [ ]:
# Run this to start the interactive interface
interface = quick_start()